# Dopant Screening Workflow

Comprehensive workflow for high-throughput dopant screening with thermodynamic stability analysis.

## Workflow Steps
1. Define screening parameters
2. Collect DFT data from Materials Project
3. Apply stability criteria (ΔE_hull ≤ 0.1 eV/atom)
4. Calculate concentration-dependent properties
5. Rank candidates by multi-objective criteria
6. Generate comprehensive reports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from ceramic_discovery.config import config
from ceramic_discovery.screening.screening_engine import ScreeningEngine
from ceramic_discovery.screening.workflow_orchestrator import WorkflowOrchestrator
from ceramic_discovery.dft.stability_analyzer import StabilityAnalyzer
from ceramic_discovery.analysis.visualizer import Visualizer

# Configuration
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Environment initialized")

## 1. Define Screening Parameters

In [ ]:
# Screening configuration
screening_config = {
    'base_systems': ['SiC', 'B4C', 'WC'],
    'dopants': [
        'Ti', 'Zr', 'Hf',  # Group 4 transition metals
        'V', 'Nb', 'Ta',   # Group 5 transition metals
        'Cr', 'Mo', 'W',   # Group 6 transition metals
        'Al', 'Y'          # Other candidates
    ],
    'concentrations': [0.01, 0.02, 0.03, 0.05, 0.10],  # 1-10% atomic
    'stability_threshold': 0.1,  # eV/atom (CORRECT threshold)
    'parallel_jobs': config.computational.max_parallel_jobs
}

print("Screening Configuration:")
print(f"  Base systems: {len(screening_config['base_systems'])}")
print(f"  Dopants: {len(screening_config['dopants'])}")
print(f"  Concentrations: {len(screening_config['concentrations'])}")
print(f"  Total combinations: {len(screening_config['base_systems']) * len(screening_config['dopants']) * len(screening_config['concentrations'])}")

## 2. Initialize Screening Engine

In [ ]:
# Initialize components
engine = ScreeningEngine()
orchestrator = WorkflowOrchestrator()
stability_analyzer = StabilityAnalyzer()

print("✓ Screening engine initialized")
print(f"  Stability threshold: {screening_config['stability_threshold']} eV/atom")
print(f"  Parallel jobs: {screening_config['parallel_jobs']}")

## 3. Run High-Throughput Screening

This cell performs the actual screening. It may take several minutes depending on the number of materials.

In [ ]:
from IPython.display import display, HTML
import time

# Create output directory
output_dir = Path('./results/dopant_screening')
output_dir.mkdir(parents=True, exist_ok=True)

# Run screening with progress tracking
print("Starting high-throughput screening...\n")
start_time = time.time()

results = orchestrator.run_batch_screening(
    config=screening_config,
    output_dir=output_dir,
    resume=False
)

elapsed_time = time.time() - start_time

print(f"\n✓ Screening complete in {elapsed_time:.1f} seconds")
print(f"  Total screened: {results['total_screened']}")
print(f"  Viable candidates: {results['viable_count']}")
print(f"  Unstable materials: {results['unstable_count']}")

## 4. Analyze Stability Results

In [ ]:
# Convert to DataFrame for analysis
df_all = pd.DataFrame(results['all_candidates'])
df_viable = pd.DataFrame(results['viable_candidates'])

print(f"Total candidates: {len(df_all)}")
print(f"Viable candidates (ΔE_hull ≤ 0.1): {len(df_viable)}")
print(f"Viability rate: {len(df_viable)/len(df_all)*100:.1f}%")

# Stability statistics
print("\nStability Statistics:")
print(df_all['energy_above_hull'].describe())

In [ ]:
# Visualize stability distribution
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Stability histogram
axes[0, 0].hist(df_all['energy_above_hull'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(0.1, color='red', linestyle='--', linewidth=2, label='Stability threshold')
axes[0, 0].set_xlabel('Energy Above Hull (eV/atom)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('Thermodynamic Stability Distribution')
axes[0, 0].legend()

# 2. Stability by base system
df_all.boxplot(column='energy_above_hull', by='base_system', ax=axes[0, 1])
axes[0, 1].axhline(0.1, color='red', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Base System')
axes[0, 1].set_ylabel('Energy Above Hull (eV/atom)')
axes[0, 1].set_title('Stability by Base System')
plt.sca(axes[0, 1])
plt.xticks(rotation=45)

# 3. Stability by dopant
dopant_stability = df_all.groupby('dopant')['energy_above_hull'].mean().sort_values()
axes[1, 0].barh(range(len(dopant_stability)), dopant_stability.values)
axes[1, 0].set_yticks(range(len(dopant_stability)))
axes[1, 0].set_yticklabels(dopant_stability.index)
axes[1, 0].axvline(0.1, color='red', linestyle='--', linewidth=2)
axes[1, 0].set_xlabel('Mean Energy Above Hull (eV/atom)')
axes[1, 0].set_title('Average Stability by Dopant')

# 4. Viability rate by concentration
viability_by_conc = df_all.groupby('concentration').apply(
    lambda x: (x['energy_above_hull'] <= 0.1).sum() / len(x) * 100
)
axes[1, 1].plot(viability_by_conc.index * 100, viability_by_conc.values, 'o-', linewidth=2, markersize=8)
axes[1, 1].set_xlabel('Dopant Concentration (%)')
axes[1, 1].set_ylabel('Viability Rate (%)')
axes[1, 1].set_title('Viability Rate vs Concentration')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / 'stability_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 5. Rank Viable Candidates

Multi-objective ranking considering stability, predicted performance, and cost.

In [ ]:
# Calculate composite ranking score
df_viable['stability_score'] = 1 - (df_viable['energy_above_hull'] / 0.1)
df_viable['performance_score'] = (df_viable['predicted_v50'] - df_viable['predicted_v50'].min()) / \
                                  (df_viable['predicted_v50'].max() - df_viable['predicted_v50'].min())

# Composite score (weighted)
df_viable['composite_score'] = (
    0.4 * df_viable['stability_score'] +
    0.4 * df_viable['performance_score'] +
    0.2 * df_viable['cost_score']  # Lower cost is better
)

# Sort by composite score
df_ranked = df_viable.sort_values('composite_score', ascending=False)

print("Top 10 Candidates:")
print(df_ranked[['composition', 'energy_above_hull', 'predicted_v50', 'composite_score']].head(10))

In [ ]:
# Visualize top candidates
top_20 = df_ranked.head(20)

fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(
    top_20['energy_above_hull'],
    top_20['predicted_v50'],
    s=top_20['composite_score'] * 500,
    c=top_20['composite_score'],
    cmap='viridis',
    alpha=0.6,
    edgecolors='black',
    linewidth=1
)

# Add labels for top 5
for idx, row in top_20.head(5).iterrows():
    ax.annotate(
        row['composition'],
        (row['energy_above_hull'], row['predicted_v50']),
        xytext=(5, 5),
        textcoords='offset points',
        fontsize=9,
        bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.5)
    )

ax.set_xlabel('Energy Above Hull (eV/atom)', fontsize=12)
ax.set_ylabel('Predicted V50 (m/s)', fontsize=12)
ax.set_title('Top 20 Dopant Candidates (size = composite score)', fontsize=14)
ax.grid(True, alpha=0.3)

cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Composite Score', fontsize=11)

plt.tight_layout()
plt.savefig(output_dir / 'top_candidates.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Property Analysis for Top Candidates

In [ ]:
# Parallel coordinates plot for top candidates
from ceramic_discovery.analysis.visualizer import Visualizer

visualizer = Visualizer()

properties = ['hardness', 'fracture_toughness', 'density', 'thermal_conductivity_1000C']
top_10 = df_ranked.head(10)

fig = visualizer.parallel_coordinates(
    data=top_10,
    properties=properties,
    color_by='composite_score',
    title='Property Comparison: Top 10 Candidates'
)

plt.savefig(output_dir / 'property_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Export Results

In [ ]:
from ceramic_discovery.reporting.data_exporter import DataExporter
from ceramic_discovery.reporting.report_generator import ReportGenerator

# Export data
exporter = DataExporter()
exporter.export_to_csv(df_ranked, output_dir / 'ranked_candidates.csv')
exporter.export_to_json(results, output_dir / 'screening_results.json')

# Generate report
report_gen = ReportGenerator()
report_gen.generate_screening_report(
    results=results,
    output_file=output_dir / 'screening_report.pdf'
)

print("✓ Results exported:")
print(f"  CSV: {output_dir / 'ranked_candidates.csv'}")
print(f"  JSON: {output_dir / 'screening_results.json'}")
print(f"  Report: {output_dir / 'screening_report.pdf'}")

## 8. Summary and Recommendations

In [ ]:
print("=" * 60)
print("SCREENING SUMMARY")
print("=" * 60)
print(f"\nTotal materials screened: {results['total_screened']}")
print(f"Viable candidates (ΔE_hull ≤ 0.1): {results['viable_count']}")
print(f"Viability rate: {results['viable_count']/results['total_screened']*100:.1f}%")

print("\nTop 5 Recommendations for Experimental Synthesis:")
for i, (idx, row) in enumerate(df_ranked.head(5).iterrows(), 1):
    print(f"\n{i}. {row['composition']}")
    print(f"   Stability: {row['energy_above_hull']:.3f} eV/atom")
    print(f"   Predicted V50: {row['predicted_v50']:.1f} m/s")
    print(f"   Composite Score: {row['composite_score']:.3f}")
    print(f"   Confidence: {row['prediction_confidence']:.2f}")

print("\n" + "=" * 60)
print("Next Steps:")
print("  1. Review top candidates with domain experts")
print("  2. Validate DFT predictions with higher-level calculations")
print("  3. Prioritize synthesis based on cost and availability")
print("  4. Design experimental validation protocol")
print("=" * 60)